# PEP-695: Type Parameter Syntax

Changes has been made to the syntax to make typing in python feel more first class. This include removing the need to import and explicitly declare a `TypeVar` or a `TypeVarTuple`.

### The new `TypeAlias`

Let's start with `TypeAlias`, a type alias can contain a `TypeVar` likes so:

In [2]:
from typing import TypeAlias, TypeVar, assert_type

T = TypeVar("T")

OldPair: TypeAlias = tuple[T, T]


def use_pair_old(pair: OldPair[int]) -> None:
    ...


use_pair_old((1, 3))


The new syntax completely removes the need for the `TypeVar`

In [3]:
type NewPair[T] = tuple[T, T]


def use_pair(pair: NewPair[int]) -> None:
    ...


use_pair_old((1, 3))


Syntax also added for more complex situations:

##### TypeVarTuple

In [4]:
from typing import Iterable, TypeVarTuple


# OLD
Ts = TypeVarTuple("Ts")
QueryResultOld: TypeAlias = Iterable[tuple[*Ts]]

# New
type QueryResult[*Ts] = Iterable[tuple[*Ts]]  

##### ParamSpec

In [5]:
from typing import Callable, ParamSpec


P = ParamSpec("P")

OldCastFunction: TypeAlias = Callable[[Callable[P, float]], Callable[P, int]]

type CastFunction[**P] = Callable[[Callable[P, float]], Callable[P, int]]

##### Bound/Constraints

In [6]:
from decimal import Decimal
from typing import Callable, Hashable, TypeAlias, TypeVar


# Bound
T = TypeVar("T", bound=Hashable)

HashFunctionOld: TypeAlias = Callable[[T], int]


type HashFunction[T: Hashable] = Callable[[T], int]


# Constraints
T = TypeVar("T", Decimal, int, float)

UnitConverterOld: TypeAlias = Callable[[T], T]

type UnitCoverter[T: (Decimal, int, float)] = Callable[[T], T] 


### Function Type Parameter

New syntax will work in generic functions, including using the bound/constraint syntax

In [7]:
from typing import Iterable, TypeVar, reveal_type

T = TypeVar("T", int, float, str)

def old_first(it: Iterable[T]) -> T | None:
    for element in it:
        return element
    
    return None


reveal_type(old_first(['a']))


def first[T: (int, float, str)](it: Iterable[T]) -> T | None:
    for element in it:
        return element
    
    return None

reveal_type(first(['a']))

Runtime type is 'str'
Runtime type is 'str'


'a'

### `Generics`

Likewise with generics:

In [8]:
from dataclasses import dataclass
from typing import Generic

T = TypeVar("T")


@dataclass
class OldMatix(Generic[T]):
    data: list[list[T]]

    def __getitem__(self, location: tuple[int, int]) -> T:
        return self.data[location[0]][location[1]]
    

@dataclass
class Matix[T]:
    data: list[list[T]]

    def __getitem__(self, location: tuple[int, int]) -> T:
        return self.data[location[0]][location[1]]
    

reveal_type(Matix([[1]]))

Runtime type is 'Matix'


Matix(data=[[1]])

Note: Type var syntax introduces annotation scopes similar to function scopes. https://docs.python.org/3.12/reference/executionmodel.html#annotation-scopes

# PEP 698: Override Decorator for Static Typing

This is something important in Java, in Python it depends on how you write your code. 

`ABC` and `Protocol`s already do this if you're writing 'interface style classes', but if you love inheritance for some reason you can use this to check that you're overriding correctly.


In [11]:
from typing import override  # type: ignore


class BaseClient:
    def send(self, values: dict) -> None:
        ...


class MyClient(BaseClient):
    @override  # Errors
    def spend(self, values: dict) -> None:
        ...


# PEP 692 – Using TypedDict for more precise **kwargs typing

In [1]:
from typing import NotRequired, TypedDict, Unpack, assert_type
from uuid import UUID


class Metadata(TypedDict):
    sender: str
    request_id: NotRequired[UUID]


def handler(data: dict, **metadata: Unpack[Metadata]) -> None:
    assert_type(metadata["sender"], str)


# Minor Typing Changes
These changes are minor and don't have a `PEP` associated.

#### [`typing.get_original_bases`](https://docs.python.org/3.12/library/types.html#types.get_original_bases)

We can now introspect the runtime generic type bases of a class, this is likely driven by Pydantic and FastAPI

In [12]:
from types import get_original_bases
from typing import Iterable


class NatualValues(Iterable[int]):
    def __iter__(self):
        n = 1
        while True:
            yield n
            n += 1


get_original_bases(NatualValues)

(typing.Iterable[int],)

#### `array.array` is now a generic type

In [1]:
from array import array

a: array[int] = array('i', [1, 2])